Perform the heatmap generation for all the CSV files provided by *Deltares* and create
* figures showing the data positions and the heatmap (pseudo-color plot)
* netCDF files (one per species) storing the heatmap.

`doplot` can be turned to 'false' to avoid creating the figures.

In [13]:
using DIVAnd
using PyPlot
using Proj4
using DelimitedFiles
using PyCall
using Dates
using NCDatasets
include("../scripts/PhytoInterp.jl")
include("../scripts/PhytoInterpPlot.jl")
doplot = true

true

## Packages for plotting

In [2]:
if doplot
    ccrs = pyimport("cartopy.crs")
    gridliner = pyimport("cartopy.mpl.gridliner")
    cfeature = pyimport("cartopy.feature")
    mticker = pyimport("matplotlib.ticker")
    myproj = ccrs.PlateCarree()
    coast = cfeature.GSHHSFeature(scale="full");
    mpl = pyimport("matplotlib");
    cartopyticker = pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
end 

PyObject <cartopy.mpl.ticker.LatitudeFormatter object at 0x7fb55e6a0bd0>

## Files and directories

In [11]:
# csvdir = "/data/EMODnet/Biology/phytoplankton/csv/" # North Sea 
csvdir = "/data/EMODnet/Biology/phytoplankton/GreaterNorthSea/csv_files/" #Greater North Sea

datadir = "../data/"
datafilelist = readdir(csvdir);
@info("Working on $(length(datafilelist)) files");
figdir = "../product/figures/heatmap/greaterNorthSea/" 
binfigdir = "../product/figures/bins/greaterNorthSea/"
datafigdir = "../product/figures/data/greaterNorthSea/"
outputdir = "../product/netCDF/greaterNorthSea/Phytoplankton/V1/"
isdir(datadir) ? " " : mkpath(datadir)
isdir(datafigdir) ? " " : mkpath(datafigdir)
isdir(figdir) ? " " : mkpath(figdir);
isdir(binfigdir) ? " " : mkpath(binfigdir);
isdir(outputdir) ? " " : mkpath(outputdir);

┌ Info: Working on 200 files
└ @ Main In[11]:6


## Prepare mask
### Interpolation grid

In [4]:
# North Sea
#longrid = -2.:0.1:10.
#latgrid = 51.:0.1:56.

# Greater North Sea
longrid = -16.:0.1:9.
latgrid = 45.:0.1:66.

45.0:0.1:66.0

### Download file

In [5]:
bathname = joinpath(datadir, "gebco_30sec_4.nc")
if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/RSwm4HPHImdZoQP/download", bathname)
else
    @info("Bathymetry file already downloaded")
end

┌ Info: Bathymetry file already downloaded
└ @ Main In[5]:5


### Read bathymetry

In [6]:
bx, by, b = load_bath(bathname, true, longrid, latgrid)
@show size(b)

size(b) = (251, 211)


(251, 211)

In [7]:
if doplot
    plot_bathymetry(bx, by, b, joinpath(figdir, "northsea_bathy");
                    domain=[-16., 9., 45., 66.], dlon=5., dlat=5.)
end

### Metrics & mask

In [8]:
mask, (pm, pn),(xi, yi) = DIVAnd.DIVAnd_rectdom(longrid, latgrid);
xi, yi, mask = DIVAnd.load_mask(bathname, true, longrid, latgrid, 0.0);
xx, yy = ndgrid(xi, yi);
if doplot
    plot_mask(bx, by, mask, joinpath(figdir, "northsea_mask"); 
              domain=[-16., 9., 45., 66.], dlon=5., dlat=3.)
end

## Main loop

In [16]:
include("../scripts/PhytoInterpPlot.jl")

plot_bincounts

In [17]:
for datafile in datafilelist[1:3]
    speciesname = get_species_name(basename(datafile))
    speciesslug = get_species_slug(basename(datafile))
    @info("Working on $(speciesname)")
    @info(speciesslug)
        
    # Data reading
    dates, lons, lats, occurs = read_data_phyto(joinpath(csvdir, datafile));
    
    # Conversion of coordinates
    # We have to go from `EPSG:32361` to `WGS84`.         
    lon, lat = transform_coords(lons, lats)
    
    # Plot presence/absence
    if doplot
        @info("Plotting")
        make_plot_presence_absence(lon, lat, occurs, 
            joinpath(datafigdir, "$(speciesslug)_data_presence_absence.png"), 
            [-16., 9., 45., 66.], 4., 2.)
    end
    
    # Create bins
    obscount, obscountlog = count_obs(longrid, latgrid, lon, lat);
    if doplot
        figname = joinpath(binfigdir, "$(speciesslug)_bins.png")
        plot_bincounts(longrid, latgrid, mask, obscountlog, speciesname, figname; 
                       domain=[-16., 9., 45., 66.], dlon=4., dlat=2., vmax=3.)
    end
    
    
    data_presence = occurs .== 1;
    data_absence = .!(data_presence);
    npre = sum(data_presence)
    nabs = sum(data_absence);
    
    @info("Presence data: $(npre), absence data: $(nabs)")
    
    # Compute heatmap
    inflation = ones(length(lon));
    
    #@time dens1, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
    #    (lon, lat), inflation, 0.1);
    @time dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_presence], lat[data_presence]), inflation[data_presence], 0.5);
    @time dens3, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_absence], lat[data_absence]), inflation[data_absence], 0.5);
    
    reldens = npre .* dens2 ./ (npre .* dens2 .+ nabs .* dens3);
    
    # Compute error field with CPME
    cpme = DIVAnd_cpme(mask, (pm, pn), (xx, yy), (lon, lat), ones(length(lon)), 0.5, 10.);
    
    if doplot
        
        
        plot_heatmap(longrid, latgrid, dens2, lon, lat, occurs,
            "$(speciesname): heatmap with 'presence' data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_presence.png"), 0., 0.2,
            domain=[-16., 9., 45., 66.], dlon=4., dlat=2.)
        
        
        plot_heatmap(longrid, latgrid, dens3, lon, lat, occurs,
            "$(speciesname): heatmap with 'absence' data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_absence.png"), 0., 0.2,
            domain=[-16., 9., 45., 66.], dlon=4., dlat=2.)
        
        """
        plot_heatmap(longrid, latgrid, dens3, lon, lat, occurs,
            "$(speciesname): heatmap with all data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_all.png"), 0., 1.,
            domain=[-16., 9., 45., 66.], dlon=4., dlat=2.)
        """
        
        plot_heatmap(longrid, latgrid, reldens, lon, lat, occurs,
            "$(speciesname) probability", 
            joinpath(figdir, "$(speciesslug)_heatmap_relative.png"), 0., 1.,
            domain=[-16., 9., 45., 66.], dlon=4., dlat=2.)
    end
    
    # Write in netCDF
    create_nc_results(joinpath(outputdir, "$(speciesslug)_heatmap.nc"), 
        longrid, latgrid, reldens, speciesname);
    
    # Add the error field
    write_nc_error(joinpath(outputdir, "$(speciesslug)_heatmap.nc"), cpme);
    
    figname = joinpath(figdir, "$(speciesslug)_cpme.png")
    @info("Saving image as $(figname)")
    
    if doplot
        plot_error(longrid, latgrid, cpme, "$(speciesname)", 
            joinpath(figdir, "$(speciesslug)_cpme.png"),
            domain=[-16., 9., 45., 66.], dlon=4., dlat=2.)
    end    
    
end

┌ Info: Working on Actinocyclus senarius
└ @ Main In[17]:4
┌ Info: Actinocyclus_senarius
└ @ Main In[17]:5
┌ Info: Plotting
└ @ Main In[17]:16
┌ Info: Presence data: 5068, absence data: 14253
└ @ Main In[17]:36


 98.926216 seconds (452.06 k allocations: 32.182 GiB, 1.60% gc time)


┌ Warning: Data array size (14253,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/ZfEqh/src/DIVAnd_heatmap.jl:48


  3.359906 seconds (446.86 k allocations: 1.644 GiB, 1.27% gc time)


┌ Info: Saving image as ../product/figures/heatmap/greaterNorthSea/Actinocyclus_senarius_cpme.png
└ @ Main In[17]:88
┌ Info: Working on Actinocyclus
└ @ Main In[17]:4
┌ Info: Actinocyclus
└ @ Main In[17]:5
┌ Info: Plotting
└ @ Main In[17]:16
┌ Info: Presence data: 5688, absence data: 27339
└ @ Main In[17]:36


...

┌ Info: Saving image as ../product/figures/heatmap/greaterNorthSea/Actinocyclus_cpme.png
└ @ Main In[17]:88
┌ Info: Working on Actinoptychus
└ @ Main In[17]:4
┌ Info: Actinoptychus
└ @ Main In[17]:5
┌ Info: Plotting
└ @ Main In[17]:16
┌ Info: Presence data: 2416, absence data: 77330
└ @ Main In[17]:36


 39.558974 seconds (216.51 k allocations: 15.536 GiB, 2.22% gc time)


┌ Warning: Data array size (77330,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/ZfEqh/src/DIVAnd_heatmap.jl:48


 64.532444 seconds (2.61 M allocations: 20.998 GiB, 0.85% gc time)


┌ Info: Saving image as ../product/figures/heatmap/greaterNorthSea/Actinoptychus_cpme.png
└ @ Main In[17]:88
